# MDE calculation template - Onprem & GBQ metrics

## Imports

In [1]:
from exp_dwh_client.core.gbq.gbq_query_executor import GBQQueryExecutor
from exp_dwh_client import DWHClient

from expmetrics.models.stats.summary_statistics import SummaryStatistics
from expmetrics.statistical_methods.mde import calculate_minimum_detectable_effect

import datetime
import pandas as pd
import pytz
import numpy as np

In [2]:
gbq_query_executor = GBQQueryExecutor()

gbq_client = DWHClient(gbq_query_executor)

## Functions

based of the example in exp metrics repository

In [3]:
def get_apply_until(days_from_today=5):
    current_utc_time = datetime.datetime.now(pytz.utc)
    two_hours_ahead_zone = pytz.FixedOffset(120)  # 120 minutes = 2 hours
    current_local_time = current_utc_time.astimezone(two_hours_ahead_zone)
    apply_until = current_local_time.replace(hour=23, minute=59, second=59, microsecond=0)
    apply_until = apply_until - datetime.timedelta(days=days_from_today)
    return apply_until


def get_apply_from(apply_until, weeks_diff=1):
    days_diff = weeks_diff * 7 - 1
    apply_from = apply_until - datetime.timedelta(days=days_diff)
    apply_from = apply_from.replace(hour=0, minute=0, second=0, microsecond=0)
    return apply_from


def rename_stats_results(df, metric):
    return df.rename(
        columns={
            metric.sample_size_alias: "sample_size",
            metric.metric_average_alias: "metric_average",
            metric.standard_deviation_alias: "standard_deviation",
        }
    )


def add_mde(df, metric, number_of_variants, alpha=0.05, beta=0.2, distribution=100):
    for j, row in df.iterrows():
        assert distribution <= 100, "Distribution should be less than or equal to 100"
        assert distribution > 1, "Distribution should be a number between 1 and 100"

        min_variant_sample_size = int(row["sample_size"] / number_of_variants * distribution / 100)
        df.loc[j, "Variant sample size"] = min_variant_sample_size
        df.loc[j, "sample_size"] = int(row["sample_size"] * distribution / 100)

        control_summary = SummaryStatistics(
            sample_size=min_variant_sample_size,
            metric_average=row["metric_average"],
            standard_deviation=row["standard_deviation"],
        )
        rmde = calculate_minimum_detectable_effect(
            metric_type=metric.metric_type,
            observed_samples={
                "control_sample_size": control_summary.sample_size,
            },
            control_summary=control_summary,
            alpha=alpha,  # probability of false positive (type I error)
            beta=beta,  # probability of false negative (type II error)
        )

        df.loc[j, "Abs MDE"] = rmde
        if control_summary.metric_average == 0:
            df.loc[j, "Rel MDE (%)"] = np.nan
        else:
            df.loc[j, "Rel MDE (%)"] = rmde / float(control_summary.metric_average) * 100
    return df


async def get_mde_df(
    metric,
    number_of_variants=2,
    alpha=0.05,
    beta=0.2,
    distribution=100,
    apply_until=None,
    max_weeks_diff=2,
    country_code=None,
    custom_country_filter=None,
):
    if apply_until is None:
        apply_until = get_apply_until(metric.lag_in_days + 5)
    if custom_country_filter is not None:
        # ignore supplied country_code and set it to LT instead
        country_code = "LT"
    df_combined = pd.DataFrame()
    for weeks_diff in range(1, max_weeks_diff + 1):
        apply_from = get_apply_from(apply_until, weeks_diff=weeks_diff)
        metric_stats_query = metric.aggregate_stats_query_v2(
            apply_from=apply_from, apply_until=apply_until, country_code=country_code
        ).get_sql(quote_char="`")
        if custom_country_filter is not None:
            metric_stats_query = metric_stats_query.replace("='LT'", f" {custom_country_filter} ")
            metric_stats_query = metric_stats_query.replace(
                " = 'LT'", f" {custom_country_filter} "
            )
        df = await GBQQueryExecutor().execute_query(metric_stats_query)

        df["experiment duration in weeks"] = weeks_diff
        df["metric"] = metric.user_friendly_name
        df = rename_stats_results(df, metric)
        df = add_mde(
            df,
            metric,
            number_of_variants=number_of_variants,
            alpha=alpha,
            beta=beta,
            distribution=distribution,
        )
        df_combined = pd.concat([df_combined, df], ignore_index=True)

    if country_code is not None and custom_country_filter is None:
        df_combined["country code"] = country_code
    elif custom_country_filter is not None:
        df_combined["country code"] = custom_country_filter
    else:
        df_combined["country code"] = "[All]"

    return df_combined


## Set up

In [4]:
APPLY_UNTIL = datetime.datetime(2025, 6, 15, 0, 0, 0, tzinfo=datetime.timezone.utc)

In [5]:
from expmetrics.metrics.gbq.metric_registry import MetricRegistry
from expmetrics.datasets.gbq import Payouts

In [7]:
metric_registry = MetricRegistry()

In [88]:
from pypika import Criterion
from expmetrics.datasets.dataset_definition_params import DatasetDefinitionParams
from expmetrics.filters.filters.gbq.date_filters import time_range_filter


class PayoutsLocal(Payouts):
    def build_filters(self, params: DatasetDefinitionParams) -> Criterion:
        filter_builder = self.init_filter_builder(params).with_time_range_filter(
            self.dimension_aliases["utc_unixtime"], filter_function=time_range_filter
        )

        if params.filter_params.country_code:
            filter_builder = filter_builder.with_test_country_filter("payouts_country_code")

        filter_builder._custom_filters.append(
            self.dimensions["payouts_amount_eur"] >=500 
        )

        filter_builder._custom_filters.append(
            self.dimensions["payouts_country_code"].isin([
    'DE',  # Germany - 1330
    'AT',  # Austria - 98
    'IT',  # Italy - 4341
    'NL',  # Netherlands - 1362
    'BE',  # Belgium - 924
    'FR',  # France - 8675
    'ES',  # Spain - 1546
    'PT',  # Portugal - 443
    'LU',  # Luxembourg - 68
    'UK',  # United Kingdom - 6711 
    'IE'   # Ireland - 65
    ] ) )
        filters = filter_builder.build()
        return Criterion.all(filters)


In [100]:
from pypika import Field
metric = metric_registry.get_by_alias("sellers")
metric.segment_dataset = PayoutsLocal()
metric.segment_dataset.dimensions["payouts_country_code"] = Field("user_country_code", alias="payouts_country_code")
await get_mde_df(metric, apply_until=APPLY_UNTIL, max_weeks_diff=5)

,metric_average,standard_deviation,sample_size,experiment duration in weeks,metric,Variant sample size,Abs MDE,Rel MDE (%),country code
0,0.695486,0.460219,13336,1,Sellers,6668.0,0.022087,3.175794,[All]
1,0.829963,0.375672,26706,2,Sellers,13353.0,0.012684,1.528292,[All]
2,0.890760,0.311944,39436,3,Sellers,19718.0,0.008646,0.970597,[All]
3,0.918407,0.273746,51132,4,Sellers,25566.0,0.006655,0.724613,[All]
4,0.935756,0.245189,61718,5,Sellers,30859.0,0.005420,0.579168,[All]


In [101]:
metric = metric_registry.get_by_alias("transactions_sold")
metric.segment_dataset = PayoutsLocal()
metric.segment_dataset.dimensions["payouts_country_code"] = Field("user_country_code", alias="payouts_country_code")
await get_mde_df(metric, apply_until=APPLY_UNTIL, max_weeks_diff=5)

,metric_average,standard_deviation,sample_size,experiment duration in weeks,metric,Variant sample size,Abs MDE,Rel MDE (%),country code
0,9.841932,29.827504,13336,1,Transactions Sold,6668.0,1.447231,14.704749,[All]
1,25.393694,98.820012,26706,2,Transactions Sold,13353.0,3.388241,13.342845,[All]
2,41.673319,196.712346,39436,3,Transactions Sold,19718.0,5.550333,13.318672,[All]
3,55.842075,294.191064,51132,4,Transactions Sold,25566.0,7.289822,13.054354,[All]
4,71.290742,407.699891,61718,5,Transactions Sold,30859.0,9.195355,12.898386,[All]


In [97]:
metric = metric_registry.get_by_alias("gmv_sold")
metric.segment_dataset = PayoutsLocal()
metric.segment_dataset.dimensions["payouts_country_code"] = Field("user_country_code", alias="payouts_country_code")
await get_mde_df(metric, apply_until=APPLY_UNTIL, max_weeks_diff=5)

,metric_average,standard_deviation,sample_size,experiment duration in weeks,metric,Variant sample size,Abs MDE,Rel MDE (%),country code
0,432.731238752,1896.934132,13336,1,GMV Sold,6668.0,92.039297,21.269390,[All]
1,1096.280140792,5212.289011,26706,2,GMV Sold,13353.0,178.713722,16.301830,[All]
2,1733.926417740,8995.175044,39436,3,GMV Sold,19718.0,253.803159,14.637482,[All]
3,2311.613756943,13130.580047,51132,4,GMV Sold,25566.0,325.365390,14.075249,[All]
4,2933.190573739,18167.768682,61718,5,GMV Sold,30859.0,409.759945,13.969769,[All]


In [102]:
metric = metric_registry.get_by_alias("listings_per_lister")
metric.segment_dataset = PayoutsLocal()
metric.segment_dataset.dimensions["payouts_country_code"] = Field("user_country_code", alias="payouts_country_code")
await get_mde_df(metric, apply_until=APPLY_UNTIL, max_weeks_diff=5)

,metric_average,standard_deviation,sample_size,experiment duration in weeks,metric,Variant sample size,Abs MDE,Rel MDE (%),country code
0,37.269582,163.293983,8439,1,Listings per Lister,4219.0,9.960571,26.725740,[All]
1,77.635930,419.753715,20323,2,Listings per Lister,10161.0,16.498526,21.251148,[All]
2,118.256186,727.496273,32410,3,Listings per Lister,16205.0,22.642537,19.147021,[All]
3,155.244519,1092.800155,43698,4,Listings per Lister,21849.0,29.291648,18.868072,[All]
4,196.500674,1698.051821,54181,5,Listings per Lister,27090.0,40.875707,20.801815,[All]


In [103]:
metric = metric_registry.get_by_alias("buyers")
metric.segment_dataset = PayoutsLocal()
metric.segment_dataset.dimensions["payouts_country_code"] = Field("user_country_code", alias="payouts_country_code")
await get_mde_df(metric, apply_until=APPLY_UNTIL, max_weeks_diff=5)

,metric_average,standard_deviation,sample_size,experiment duration in weeks,metric,Variant sample size,Abs MDE,Rel MDE (%),country code
0,0.210258,0.407507,13336,1,Buyers,6668.0,0.019422,9.237305,[All]
1,0.311391,0.463071,26706,2,Buyers,13353.0,0.015762,5.061786,[All]
2,0.369409,0.482651,39436,3,Buyers,19718.0,0.013563,3.671673,[All]
3,0.416686,0.493015,51132,4,Buyers,25566.0,0.012189,2.925218,[All]
4,0.451197,0.497617,61718,5,Buyers,30859.0,0.011209,2.484375,[All]


In [115]:
metric = metric_registry.get_by_alias("listers_per_au")
metric.segment_dataset = PayoutsLocal()
metric.segment_dataset.dimensions["payouts_country_code"] = Field("user_country_code", alias="payouts_country_code")
await get_mde_df(metric, apply_until=APPLY_UNTIL, max_weeks_diff=5)

,metric_average,standard_deviation,sample_size,experiment duration in weeks,metric,Variant sample size,Abs MDE,Rel MDE (%),country code
0,0.632798,0.482060,13336,1,Listers per AU,6668.0,0.023219,3.669309,[All]
1,0.760990,0.426487,26706,2,Listers per AU,13353.0,0.014466,1.900927,[All]
2,0.821838,0.382654,39436,3,Listers per AU,19718.0,0.010667,1.297966,[All]
3,0.854612,0.352495,51132,4,Listers per AU,25566.0,0.008625,1.009228,[All]
4,0.877880,0.327427,61718,5,Listers per AU,30859.0,0.007288,0.830227,[All]
